In [94]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

In [95]:
df19 = pd.read_csv("./2019_april.csv")
df20 = pd.read_csv("./2020_april.csv")
df21 = pd.read_csv("./2021_april.csv")

In [96]:
features = set(df19.columns) and set(df20.columns) and set(df21.columns)
features -= {"wind-gust","precipitation","wd"}
features = list(features)
features.sort()

In [97]:
print(features)

['City', 'Date', 'co', 'dew', 'humidity', 'no2', 'o3', 'pm10', 'pm25', 'pressure', 'so2', 'temperature', 'wind-speed']


In [80]:
df19 = df19[features]
df20 = df20[features]
df21 = df21[features]

In [88]:
def interpolate(df, year):
    df.replace(0,np.nan,inplace=True)
    for i in range(18):
        df_ = df.iloc[i*30:i*30 + 30,1:]
        tndx = pd.date_range(start=year+"-04-01",end=year+"-04-30",freq='D')
        df_.index = tndx
        df_.interpolate(method = 'time',inplace=True)
        df_.iloc[:,0] = tndx
        df_.index = range(i*30,i*30 + 30)
        df.iloc[i*30:i*30 + 30,1:] = df_
    return df

In [90]:
def impute_data(df):
    data = df.iloc[:,2:]
    imputer = IterativeImputer(BayesianRidge())
    df.iloc[:,2:] = imputer.fit_transform(data)
    return df

In [92]:
df19 = interpolate(df19,"2019")
df20 = interpolate(df20,"2020")
df21 = interpolate(df21,"2021")

In [ ]:
df19 = impute_data(df19)
df20 = impute_data(df20)
df21 = impute_data(df21)

In [ ]:
df19.to_csv("./2019_april_clean.csv", index = False)
df20.to_csv("./2020_april_clean.csv", index = False)
df21.to_csv("./2021_april_clean.csv", index = False)